In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [49]:
train_df_original = pd.read_csv('TRAIN_FINAL.csv')
test_df = pd.read_csv('Test_v1.csv')

In [50]:
positive_dataset = train_df_original[train_df_original['TARGET']==0]

In [51]:
negative_dataset = train_df_original[train_df_original['TARGET']==1]

In [93]:
sample_positive = positive_dataset.sample(n=1763*10,random_state=42)

In [94]:
train_df = pd.concat([sample_positive,negative_dataset])

In [95]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19393 entries, 41644 to 45606
Columns: 370 entries, col_0 to TARGET
dtypes: float64(111), int64(259)
memory usage: 54.9 MB


In [96]:
train_df = train_df.sample(frac = 1,random_state=42)

In [32]:
train_df1 = train_df[non_zero]

In [97]:
X = train_df.drop(['TARGET'],axis=1)

y = train_df['TARGET']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42,stratify = y)

In [41]:
import lightgbm as lgb 

lgb_params = {"objective" : "binary", "metric" : ['auc'],
              "num_leaves" : 8, "learning_rate" : 0.05, 
              "is_unbalance": True,
              "bagging_fraction" : 0.5, "feature_fraction" : 0.8, "bagging_frequency" : 9,
               }
    
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_val = lgb.Dataset(X_test, label=y_test)
model = lgb.train(lgb_params, lgb_train, 2000, valid_sets=[lgb_val], verbose_eval=100)

[100]	valid_0's auc: 0.830874
[200]	valid_0's auc: 0.828176
[300]	valid_0's auc: 0.824905
[400]	valid_0's auc: 0.820848
[500]	valid_0's auc: 0.817815
[600]	valid_0's auc: 0.813503
[700]	valid_0's auc: 0.809724
[800]	valid_0's auc: 0.805818
[900]	valid_0's auc: 0.80299
[1000]	valid_0's auc: 0.799967
[1100]	valid_0's auc: 0.796306
[1200]	valid_0's auc: 0.795081
[1300]	valid_0's auc: 0.793085
[1400]	valid_0's auc: 0.791045
[1500]	valid_0's auc: 0.789665
[1600]	valid_0's auc: 0.788703
[1700]	valid_0's auc: 0.786102
[1800]	valid_0's auc: 0.785364
[1900]	valid_0's auc: 0.785119
[2000]	valid_0's auc: 0.784218


In [9]:
ind = test_df['index']
test = test_df.drop(['index'],axis=1)

In [34]:
pred_val = model.predict(test,num_iteration=model.best_iteration)

y_out = [1 if i > 0.5 else 0 for i in pred_val]

In [35]:
dfdict={}

dfdict["index"]=ind
dfdict["TARGET"]=y_out

df=pd.DataFrame(dfdict)
df.to_csv("solution.csv",index=False,columns=["index","TARGET"])

In [36]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=200)
rf.fit(X_train,y_train)

pred = rf.predict(X_test)
print(roc_auc_score(y_test,pred))

0.729381443299


In [36]:
y_out = rf.predict(test)

In [37]:
dfdict={}

dfdict["index"]=ind
dfdict["TARGET"]=y_out

df=pd.DataFrame(dfdict)
df.to_csv("solution.csv",index=False,columns=["index","TARGET"])

In [114]:
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=100,max_depth=3,scale_pos_weight=10)
xgb.fit(X_train,y_train)

pred = xgb.predict(X_test)
print(roc_auc_score(y_test,pred))

0.779427868719


In [ ]:
'''17630 positive points, xgb 100est 3depth pos_wt 10 0.779427868719'''

In [ ]:
'''100-0.768 max depth 2'''

In [115]:
y_out = xgb.predict(test)

In [116]:
dfdict={}

dfdict["index"]=ind
dfdict["TARGET"]=y_out

df=pd.DataFrame(dfdict)
df.to_csv("solution.csv",index=False,columns=["index","TARGET"])

In [24]:
print(X_train.columns)

Index(['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7',
       'col_8', 'col_9',
       ...
       'col_359', 'col_360', 'col_361', 'col_362', 'col_363', 'col_364',
       'col_365', 'col_366', 'col_367', 'col_368'],
      dtype='object', length=369)


In [23]:
print(xgb.feature_importances_)

[ 0.01030928  0.16494845  0.00687285  0.02061856  0.00687285  0.          0.
  0.          0.          0.          0.00687285  0.0137457   0.00687285
  0.02061856  0.00687285  0.          0.          0.02749141  0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0

In [26]:
def get_non_zero_importance(cols,xgb):
    
    non_zero = []
    for i,val in enumerate(xgb.feature_importances_):
        if(val>0.00001):
            non_zero.append(cols[i])
    return non_zero        
    
    

In [27]:
non_zero = get_non_zero_importance(X_train.columns,xgb)

In [31]:
non_zero.append('TARGET')